In [1]:
from huggingface_hub import login

login()

In [14]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np

# 사용할 모델 이름
model_name = "meta-llama/Llama-3.2-3B"

# 토크나이저와 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()  # 평가 모드로 전환

# 추출할 숫자 문자열 리스트 구성
numbers = (
    [str(i) for i in range(10)]          # "0" ~ "9"
    + [f"{i:02d}" for i in range(100)]   # "00" ~ "99"
    + [f"{i:03d}" for i in range(1000)]  # "000" ~ "999"
)

# 임베딩 벡터를 저장할 dictionary
embedding_dict = {}

# GPU 사용 가능 시 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for num in numbers:
    # 입력 토큰화
    inputs = tokenizer(num, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)

    # outputs.last_hidden_state: (batch_size, sequence_length, hidden_size)
    # 여기서는 전체 시퀀스의 임베딩을 평균 풀링(Mean Pooling)해서 사용
    hidden_states = outputs.last_hidden_state  # shape: [1, seq_len, hidden_size]
    embedding_vector = hidden_states.mean(dim=1).squeeze().cpu().numpy()

    # embedding_dict에 {문자열: 임베딩 벡터} 형태로 저장
    embedding_dict[num] = embedding_vector

# 임베딩 사전을 npz 파일로 저장
np.savez("llama_embeddings.npz", **embedding_dict)

print("임베딩 벡터가 embeddings.npz 파일로 저장되었습니다.")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
import os
import numpy as np

print("현재 디렉토리:", os.getcwd())
print("현재 디렉토리 목록:", os.listdir("."))

file_path = "llama_embeddings.npz"
if not os.path.exists(file_path):
    print(f"파일 {file_path} 이(가) 존재하지 않습니다!")
else:
    data = np.load(file_path)
    print("데이터 키 목록:", data.files)
    if "005" in data.files:
        embedding_005 = data["005"]
        print("005 임베딩:", embedding_005)
    else:
        print("'005'라는 키가 없습니다!")
